<a href="https://colab.research.google.com/github/DimosAndronoudis/Music_ChitChat/blob/main/Conversational_MusicQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch pandas --quiet

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import pandas as pd
import torch

In [ ]:
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


DATA_PATH = "music_qa_dataset.csv"  # Replace with your actual path
music_df = pd.read_csv(DATA_PATH)


split_index = int(len(music_df) * 0.9)
train_df = music_df.iloc[:split_index]
val_df = music_df.iloc[split_index:]


train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))

In [ ]:

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=1,
    save_strategy="epoch",
    report_to="none",
    disable_tqdm=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
# Train
print("\n📚 Fine-tuning Phi-1.5 on external music knowledge dataset...")
trainer.train()

# Save
model.save_pretrained("./phi_finetuned")
tokenizer.save_pretrained("./phi_finetuned")

# Load
chat_tokenizer = AutoTokenizer.from_pretrained("./phi_finetuned")
chat_model = AutoModelForCausalLM.from_pretrained("./phi_finetuned")


In [ ]:

def chat():
    print("\n Music bot is ready. Type 'exit' to stop.")
    while True:
        prompt = input("\nYou: ")
        if prompt.lower() == "exit":
            print(" Goodbye!")
            break
        input_ids = chat_tokenizer.encode(prompt, return_tensors="pt")
        output = chat_model.generate(input_ids, max_length=200, do_sample=True, temperature=0.7)
        response = chat_tokenizer.decode(output[0], skip_special_tokens=True)
        print("Bot:", response)

print("\n Training complete. Run chat() to start chatting about music!")